In [2]:
import json as j
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
json_data = None
with open('./yelp_academic_dataset_review.json') as data_file:
    lines = data_file.readlines()
    joined_lines = "[" + ",".join(lines) + "]"

    json_data = j.loads(joined_lines)

In [4]:
data = pd.DataFrame(json_data)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

X_train, X_test, y_train, y_test = train_test_split(data['cleaned'], data.stars, test_size=0.2)



In [5]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [6]:
model = pipeline.fit(X_train, y_train)

In [7]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)


In [8]:


print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.5790961680657649


In [49]:
print(model.predict(['Dr. Merriman is very nice and explained everything to me.']))


[5]


In [25]:
print(model.predict(["Dr. Jain was extremely thorough with diagnosing our sons football injury. Although the outcome wasnt what we wanted to hear he was truthful and blunt about treatment. We appreciate his kindness| concern| and care he showed our son."]))

[5]


In [24]:
print(model.predict(['Exactly one year ago today| in a random act of kindness. Someone reached out to me and offered me the use of this scooter. After 3 days of juggling food| drinks| and crutches from my kitchen back to my bed (usually catastrophica']))


[1]


In [27]:
model.predict(['Exactly one year ago today| in a random act of kindness. Someone reached out to me and offered me the use of this scooter. After 3 days of juggling food| drinks| and crutches from my kitchen back to my bed (usually catastrophica'])[0]

1

In [28]:
model.predict(['Very satisfied and feel like I am in good hands.'])[0]

4

In [12]:
model.predict(["He is neither intelligent nor handsome"])[0]

4

In [57]:
# 5 positive
# 4 somwat positive
# 3 negative
# 1<2
# 1 negative 


# worst
# bad


In [13]:
import pandas as pd

In [18]:
DATA_SETS_DOC = pd.read_csv("./file1.csv",engine="python")
DATA_SETS_DOC

,Unnamed: 0,Doctor,Review
0,0,Jarrad A. Merriman| MD| MPH,ï¿½Dr. Merriman is very nice and explained eve...
1,1,Jaime D. Hernandez| MD,ï¿½Total hip replacement: I can't believe how ...
2,2,Nickul S. Jain| MD,ï¿½Dr. Jain did surgery on my lower lumbar. I ...
3,3,Nickul S. Jain| MD,ï¿½Indeed| the most treasured of the Doctors I...
4,4,Nickul S. Jain| MD,ï¿½Dr. Jain was a kind| competent| and excelle...
...,...,...,...
782,104,Thomas E. Dudley| M.D.| Ph.D.,"ï¿½""The folks at Heartland Orthopedics were ab..."
783,105,Russell S. Sticha| D.P.M.,"ï¿½""Everyone seemed very competent at my first..."
784,106,Paul A. Dale| M.D.,"ï¿½""I find the doctor that did my knee replace..."
785,107,Dennis P. Weigel| M.D.,"ï¿½""I would like to thank Dr. Dennis Weigel fo..."


In [19]:
Review=DATA_SETS_DOC.iloc[:,-1]

In [11]:
Score_review=[]

In [20]:
for i in Review:
    Score_review.append(model.predict([i])[0])

In [21]:
len(Score_review)

896

In [22]:
import csv

TypeError: argument 1 must be an iterator

In [23]:
i=-1
with open('file1.csv','r') as csvinput:
    with open('output.csv', 'w+') as csvoutput:
        writer = csv.writer(csvoutput)
        for row in csv.reader(csvinput):
            writer.writerow(row+[Score_review[i]])
            i=i+1
            